In [1]:
import anndata
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
import json

In [2]:
t2g_file='/home/yuxuans/single_cell/t2g.txt'
all_cell_path='/nfs/turbo/umms-welchjd/yuxuans/single_cell/final/original/all_cell.h5ad'
reference_cell_path='/nfs/turbo/umms-welchjd/yuxuans/single_cell/reference/reference_anndata.h5ad'
reference_genecode_path='/home/yuxuans/alternative_splicing/as_type/file/generate_file/selected_reference_transcript_dic.json'
isoform_genecode_path='/home/yuxuans/alternative_splicing/as_type/file/generate_file/selected_isoform_transcript_dic.json'
reference_chess_path='/home/yuxuans/alternative_splicing/chess/as_type/generate_file/json/reference_position_chess.json'
isoform_chess_path='/home/yuxuans/alternative_splicing/chess/as_type/generate_file/json/isoform_position_chess.json'

In [3]:
all_cell=anndata.read(all_cell_path)

In [4]:
reference_cell=anndata.read(reference_cell_path)

/home/yuxuans/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


# reference: 
Booeshaghi, A. Sina, et al. "Isoform cell-type specificity in the mouse primary motor cortex." Nature 598.7879 (2021): 195-199.
https://github.com/pachterlab/BYVSTZP_2020

In [5]:
#add compartment,tissue,cell type information for each cell
def add_cell_feature(all_anndata,reference_anndata,feature):
    all_anndata.obs[feature]=None
    for index in range(len(all_anndata.obs)):
        cell_id=all_anndata.obs.index[index]
        if cell_id in reference_anndata.obs.index:
            #print(cell_id)
            if type(reference_anndata.obs[feature][cell_id])!=str:
                all_anndata.obs[feature][cell_id]=reference_anndata.obs[feature][cell_id][0]
                #print(all_anndata.obs[feature][cell_id])
            else:
                all_anndata.obs[feature][cell_id]=reference_anndata.obs[feature][cell_id]
        elif cell_id+'.homo.gencode.v30.ERCC.chrM' in reference_anndata.obs.index:
            if type(reference_anndata.obs[feature][cell_id+'.homo.gencode.v30.ERCC.chrM'])!=str:
                all_anndata.obs[feature][cell_id]=reference_anndata.obs[feature][cell_id+'.homo.gencode.v30.ERCC.chrM'][0]
            else:
                all_anndata.obs[feature][cell_id]=reference_anndata.obs[feature][cell_id+'.homo.gencode.v30.ERCC.chrM']
    return all_anndata

In [6]:
all_cell=add_cell_feature(all_cell,reference_cell,feature='compartment')
all_cell=add_cell_feature(all_cell,reference_cell,feature='organ_tissue')
all_cell=add_cell_feature(all_cell,reference_cell,feature='cell_ontology_class')
all_cell=add_cell_feature(all_cell,reference_cell,feature='free_annotation')

In [7]:
#only select cell with cell types
def select_annotated_cell(all_anndata):
    annotated_cell_list=[]
    for index in range(len(all_anndata.obs)):
        cell_id=all_anndata.obs.index[index]
        if all_anndata.obs['cell_ontology_class'][index]!=None:
            if cell_id not in annotated_cell_list:
                annotated_cell_list.append(cell_id)
    annotated_cell = all_anndata[annotated_cell_list,:]
    return annotated_cell
all_cell=select_annotated_cell(all_cell)

In [8]:
#add gene name, ID, transcript length
def add_gene(all_anndata,t2g_file):
    t2g=pd.read_csv(t2g_file,sep='\t',header=None,names=['gene ID','gene name','genecode_identifier','chromosome number','start position','end position','strand'])
    all_anndata.var['gene_ID']=None
    all_anndata.var['gene_name']=None
    all_anndata.var['length']=None
    for index in range(len(all_anndata.var.index)):
        transcript_id=all_anndata.var.index[index]
        all_anndata.var['gene_ID'][transcript_id]=t2g['gene ID'][transcript_id]
        all_anndata.var['gene_name'][transcript_id]=t2g['gene name'][transcript_id] 
        start_position=t2g['start position'][transcript_id]
        end_position=t2g['end position'][transcript_id]
        length=int(end_position)-int(start_position)+1
        all_anndata.var['length'][transcript_id]=str(length)
    return all_anndata
all_cell=add_gene(all_cell,t2g_file)

/tmp/ipykernel_1912092/160974847.py:4: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  all_anndata.var['gene_ID']=None


In [9]:
#add the information about the reference/isoform class
def add_class_genecode(all_anndata,index,new_isoform_genecode,new_reference_genecode):
    transcript=all_anndata.var.index[index].split('.')[0]
    if transcript in new_isoform_genecode.keys():
        all_anndata.var['class'][index]='isoform'
        all_anndata.var['uniprot'][index]=new_isoform_genecode[transcript]
    elif transcript in new_reference_genecode.keys():
        all_anndata.var['class'][index]='reference'
        all_anndata.var['uniprot'][index]=new_reference_genecode[transcript]
    return all_anndata
def add_class_chess(all_anndata,index,isoform_chess,reference_chess):
    if all_anndata.var.index[index] in isoform_chess.keys():
        all_anndata.var['class'][index]='isoform'
    elif all_anndata.var.index[index] in reference_chess.keys():
        all_anndata.var['class'][index]='reference'
    return all_anndata

In [10]:
def add_class(all_anndata,isoform_genecode_path,reference_genecode_path,isoform_chess_path,reference_chess_path):
    with open(isoform_genecode_path,'r') as load_f:
        isoform_genecode=json.load(load_f)
    with open(reference_genecode_path,'r') as load_f:
        reference_genecode=json.load(load_f)
    with open(isoform_chess_path,'r') as load_f:
        isoform_chess=json.load(load_f)
    with open(reference_chess_path,'r') as load_f:
        reference_chess=json.load(load_f)
    all_anndata.var['class']='na'
    all_anndata.var['uniprot']='na'
    new_isoform_genecode={y:x for x,y in isoform_genecode.items()}
    new_reference_genecode={y:x for x,y in reference_genecode.items()}
    for index in range(len(all_anndata.var)):
        if all_anndata.var.index[index][:3]=='CHS':
            all_anndata=add_class_chess(all_anndata,index,isoform_chess,reference_chess)
        else:
            all_anndata=add_class_genecode(all_anndata,index,new_isoform_genecode,new_reference_genecode)
    return all_anndata

In [11]:
all_cell=add_class(all_cell,isoform_genecode_path,reference_genecode_path,isoform_chess_path,reference_chess_path)

In [12]:
#seperate them into genecode and chess
def separate_transcript(all_anndata):
    genecode_list=[]
    chess_list=[]
    for index in range(len(all_anndata.var)):
        if all_anndata.var.index[index][:3]=='CHS':
            chess_list.append(all_anndata.var.index[index])
        else:
            genecode_list.append(all_anndata.var.index[index])
    genecode_anndata=all_anndata[:,genecode_list]
    chess_anndata=all_anndata[:,chess_list]
    return genecode_anndata,chess_anndata
genecode_cell,chess_cell=separate_transcript(all_cell)

In [13]:
#normalize
def normalization(anndata):
    raw = anndata.X.todense()
    anndata.var['length']=anndata.var['length'].astype(int)
    scaled = raw/anndata.var.length.values
    anndata.layers["norm"] = csr_matrix(normalize(scaled, norm='l1', axis=1)*1000000)
    anndata.layers["log1p"] = csr_matrix(np.log1p(anndata.layers["norm"]))
    return anndata

In [14]:
normed_genecode_cell=normalization(genecode_cell)

/tmp/ipykernel_1912092/2226231581.py:4: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  anndata.var['length']=anndata.var['length'].astype(int)
/home/yuxuans/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [16]:
#normed_genecode_cell.write('/nfs/turbo/umms-welchjd/yuxuans/single_cell/final/normed/normed_genecode.h5ad')

In [15]:
normed_chess_cell=normalization(chess_cell)

/tmp/ipykernel_1912092/2226231581.py:4: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  anndata.var['length']=anndata.var['length'].astype(int)
/home/yuxuans/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [17]:
#normed_chess_cell.write('/nfs/turbo/umms-welchjd/yuxuans/single_cell/final/normed/normed_chess.h5ad')

In [16]:
#subset the data where we modeled the structures
normed_swissport_cell=normed_genecode_cell[:,normed_genecode_cell.var['uniprot']!='na']

In [18]:
#save the h5ad file
#normed_swissport_cell.write('normed_swiss.h5ad')

/home/yuxuans/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/yuxuans/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/yuxuans/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/yuxuans/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/yuxuans/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/yuxuans/.local/lib/